# Preparam el notebook

In [7]:
import os
import time
import itertools

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import image
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

from skimage.feature import hog
from skimage import io, feature, filters
from skimage.transform import rotate, resize
from skimage.util import random_noise
from scipy import ndimage

# Llegim totes les imatges del dataset

In [2]:
def get_image_files_by_folder(root_folder, image_extensions=('jpg', 'jpeg', 'png', 'gif', 'bmp')):
    image_files_by_folder = {}

    # Walk through the root folder and its subdirectories
    for folder_path, _, files in os.walk(root_folder):
        folder_name = os.path.basename(folder_path)
        image_files = []

        for file in files:
            file_extension = file.lower().split('.')[-1]
            if file_extension in image_extensions:
                image_files.append(os.path.join(folder_path, file))

        if image_files:
            image_files_by_folder[folder_name] = image_files

    return image_files_by_folder

In [3]:
train_dataset_path = "data/train"
images_to_load_train = get_image_files_by_folder(train_dataset_path)
images_to_load_train.keys()

dict_keys(['bedroom', 'coast', 'forest', 'highway', 'industrial', 'insidecity', 'kitchen', 'livingroom', 'mountain', 'office', 'opencountry', 'store', 'street', 'suburb', 'tallbuilding'])

In [4]:
test_dataset_path = "data/test"
images_to_load_test = get_image_files_by_folder(test_dataset_path)
images_to_load_test.keys()

dict_keys(['bedroom', 'coast', 'forest', 'highway', 'industrial', 'insidecity', 'kitchen', 'livingroom', 'mountain', 'office', 'opencountry', 'store', 'street', 'suburb', 'tallbuilding'])

# Cream el dataframe

In [11]:
def read_and_flatten_images(image_list, augment_data):
    start_time = time.time()
    image_data = []
    
    for image_path in image_list:
        img = io.imread(image_path, as_gray=True)
        img = resize(img, (200, 200))
        if augment_data:
            flipped_img = img[:, ::-1]
            rotated_5 = rotate(img, 5)
            rotated_neg_5 = rotate(img, -5)
            images = [img, flipped_img, rotated_5, rotated_neg_5]
        else:
            images = [img]

        for image_to_process in images:
            hog_vectors = hog(image_to_process, orientations=9, pixels_per_cell=(20, 20),
                              cells_per_block=(10, 10), channel_axis=None, transform_sqrt=True)
            # TODO: Create filter bank? https://github.com/williamcfrancis/Gabor-Filters-and-SVM-on-MNIST/blob/main/Gabor_Filters_SVM_MNIST.ipynb
            freq, theta, bandwidth = 0.1, np.pi/4, 1
            real, _ = filters.gabor(image_to_process, frequency=freq, theta=theta, bandwidth=bandwidth)

            image_data.append(np.concatenate((hog_vectors, real.flatten())))
    
    print("Read + HOG + Data augmentation time + GABOR:", time.time() - start_time, "seconds")
    return image_data

def create_dataframe_from_categories(images_to_load, augment_data=True):
    df = pd.DataFrame()
    for label, image_list in images_to_load.items():
        image_data = read_and_flatten_images(images_to_load[label], augment_data)
        next_part = pd.DataFrame(image_data)
        next_part['label'] = label
        df = pd.concat([df, next_part], ignore_index=True)
    return df

In [12]:
train_df = create_dataframe_from_categories(images_to_load_train)
test_df = create_dataframe_from_categories(images_to_load_test, augment_data=False)

Read + HOG + Data augmentation time + GABOR: 16.134086847305298 seconds
Read + HOG + Data augmentation time + GABOR: 16.216979265213013 seconds
Read + HOG + Data augmentation time + GABOR: 16.798370122909546 seconds
Read + HOG + Data augmentation time + GABOR: 16.997578382492065 seconds
Read + HOG + Data augmentation time + GABOR: 17.107712984085083 seconds
Read + HOG + Data augmentation time + GABOR: 16.98561191558838 seconds
Read + HOG + Data augmentation time + GABOR: 16.574727296829224 seconds
Read + HOG + Data augmentation time + GABOR: 16.388156414031982 seconds
Read + HOG + Data augmentation time + GABOR: 16.365331411361694 seconds
Read + HOG + Data augmentation time + GABOR: 16.317786931991577 seconds
Read + HOG + Data augmentation time + GABOR: 16.33007264137268 seconds
Read + HOG + Data augmentation time + GABOR: 16.398698329925537 seconds
Read + HOG + Data augmentation time + GABOR: 16.38350820541382 seconds
Read + HOG + Data augmentation time + GABOR: 16.356008052825928 sec

In [13]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,40891,40892,40893,40894,40895,40896,40897,40898,40899,label
0,0.008185,0.002439,0.002141,0.001423,0.025780,0.068306,0.002338,0.002355,0.005720,0.010730,...,0.000578,0.000730,0.000873,0.000898,0.000848,0.000732,0.000567,0.000464,0.000404,bedroom
1,0.008833,0.017317,0.004386,0.001850,0.005067,0.001878,0.001946,0.003284,0.004102,0.031619,...,-0.001853,-0.001520,-0.000995,-0.000464,0.000027,0.000302,0.000479,0.000513,0.000496,bedroom
2,0.002579,0.000566,0.001385,0.035966,0.183858,0.043381,0.001919,0.002758,0.003688,0.003583,...,0.000021,0.000041,0.000102,0.000227,0.000407,0.000618,0.000827,0.000989,0.001078,bedroom
3,0.204673,0.006517,0.000000,0.000042,0.000014,0.000088,0.000036,0.000030,0.000021,0.007952,...,-0.011975,-0.009911,-0.006280,-0.002185,0.001247,0.003185,0.003890,0.003835,0.003695,bedroom
4,0.030251,0.001918,0.000757,0.000693,0.004884,0.003965,0.001368,0.002498,0.023177,0.035379,...,0.000788,0.000693,0.000508,0.000269,-0.000003,-0.000264,-0.000476,-0.000640,-0.000753,bedroom


In [14]:
train_df.shape

(6000, 40901)

In [15]:
test_df.shape

(2985, 40901)

# Cream els conjunts de test i d'entrenament

In [16]:
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']

X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

# Entrenam el model

In [19]:
svm = SVC(C=1, kernel='poly', random_state=33)
start_time = time.time()
svm.fit(X_train, y_train)
print("fit time:", time.time() - start_time, "seconds")

fit time: 741.6999897956848 seconds


# Donam una predicció

In [20]:
y_predicted = svm.predict(X_test)

print(f"La precisició es de: {accuracy_score(y_test, y_predicted)}")

La precisició es de: 0.36348408710217756
